# Bird Object Detection - Training

Upload the training and validation data to the S3 bucket. We do this in multiple channels. Channels are simply directories in the bucket that differentiate the types of data provided to the algorithm. For the object detection algorithm, we call these directories train and validation.

Next we define an output location in S3, where the model artifacts will be placed on completion of the training. These artifacts are the output of the algorithm’s traning job. We also get the URI to the Amazon SageMaker Object Detection docker image. This ensures the estimator uses the correct algorithm from the current region.

The object detection algorithm at its core is the Single-Shot Multi-Box detection algorithm (SSD). This algorithm uses a base_network, which is typically a VGG or a ResNet. The Amazon SageMaker object detection algorithm supports VGG-16 and ResNet-50. It also has a number of hyperparameters that help configure the training job. The next step in our training, is to setup these hyperparameters and data channels for training the model. See the SageMaker Object Detection documentation for more details on its specific hyperparameters.

In [82]:
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
# this will create a 'default' sagemaker bucket if it doesn't exist (sagemaker-region-accountid)
bucket = sagemaker_session.default_bucket()

prefix = "DEMO-ObjectDetection-birds"
TRAIN_REC_FILE = "birds_ssd_sample_train.rec"
VAL_REC_FILE = "birds_ssd_sample_val.rec"

# Upload the RecordIO files to train and validation channels
train_channel = prefix + "/train"
validation_channel = prefix + "/validation"

sess.upload_data(path="birds_ssd_sample_train.rec", bucket=bucket, key_prefix=train_channel)
sess.upload_data(path="birds_ssd_sample_val.rec", bucket=bucket, key_prefix=validation_channel)

s3_train_data = "s3://{}/{}".format(bucket, train_channel)
s3_validation_data = "s3://{}/{}".format(bucket, validation_channel)
print(s3_train_data)
print(s3_validation_data)

s3://sagemaker-eu-central-1-559317267498/DEMO-ObjectDetection-birds/train
s3://sagemaker-eu-central-1-559317267498/DEMO-ObjectDetection-birds/validation
